**Table of contents**<a id='toc0_'></a>    
- [同次変換行列を用いた順運動学](#toc1_)    
    - [pybulletの起動](#toc1_1_1_)    
    - [pybulletの初期設定](#toc1_1_2_)    
    - [ロボットアームの生成](#toc1_1_3_)    
    - [2次元平面における同次変換の定義](#toc1_1_4_)    
    - [同次変換行列による順運動学の実行](#toc1_1_5_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[同次変換行列を用いた順運動学](#toc0_)

本notebookでは、2軸ロボットアームを用いて「同次変換行列を用いた順運動学」をPybulletで実装する手順について解説します。

（pybulletで使用可能な関数がまとめられたマニュアルについては[こちら](https://github.com/bulletphysics/bullet3/blob/master/docs/pybullet_quickstartguide.pdf)を参照してください。）

<br>

同次変換行列による順運動学では、下動画のように
- 各リンクに座標系を張り付け、
- 「リンク1座標系$\Sigma_{1}$」→「リンク2座標系$\Sigma_{2}$」→　... →「エンドエフェクタ座標系$\Sigma_{\mathrm{e}}$」と順番に座標変換を繰り返すことで
- 「リンク1座標系$\Sigma_{1}$」から見た時の「エンドエフェクタ座標系$\Sigma_{\mathrm{e}}$」の位置·姿勢を求める

ことができます。

![](../images/RobotArm/robot_arm_homogeneous_matrix_forward_kinematics/overview.gif)

### <a id='toc1_1_1_'></a>[pybulletの起動](#toc0_)

In [1]:
import pybullet
import pybullet_data
physics_client = pybullet.connect(pybullet.GUI) 

pybullet build time: Nov 28 2023 23:45:17


startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=Mesa
GL_RENDERER=llvmpipe (LLVM 15.0.7, 256 bits)
GL_VERSION=4.5 (Core Profile) Mesa 23.2.1-1ubuntu3.1~22.04.2
GL_SHADING_LANGUAGE_VERSION=4.50
pthread_getconcurrency()=0
Version = 4.5 (Core Profile) Mesa 23.2.1-1ubuntu3.1~22.04.2
Vendor = Mesa
Renderer = llvmpipe (LLVM 15.0.7, 256 bits)
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started


### <a id='toc1_1_2_'></a>[pybulletの初期設定](#toc0_)

In [2]:
pybullet.resetSimulation() # シミュレーション空間をリセット
pybullet.setAdditionalSearchPath(pybullet_data.getDataPath()) # pybulletに必要なデータへのパスを追加
pybullet.setGravity(0.0, 0.0, -9.8) # 地球上における重力に設定
time_step = 1./240.
pybullet.setTimeStep(time_step) # 1stepあたりに経過する時間の設定

#床の読み込み
plane_id = pybullet.loadURDF("plane.urdf")

# GUIモードの際のカメラの位置などを設定
camera_distance = 2.0
camera_yaw = 0.0 # deg
camera_pitch = -20 # deg
camera_target_position = [0.0, 0.0, 0.0]
pybullet.resetDebugVisualizerCamera(camera_distance, camera_yaw, camera_pitch, camera_target_position)

ven = Mesa
ven = Mesa


### <a id='toc1_1_3_'></a>[ロボットアームの生成](#toc0_)
今回は、2軸ロボットアーム`simple_2d_arm.urdf`を生成します。  
ロボットは下図のような構成になっています（センサーについては、`robot_arm_sensor.ipynb`にて解説しています。）  

![](../images/RobotArm/2d_robot_arm.png)

In [3]:
# ロボットの読み込み
arm_start_pos = [0, 0, 0.1]  # 初期位置(x,y,z)を設定
arm_start_orientation = pybullet.getQuaternionFromEuler([0,0,0])  # 初期姿勢(roll, pitch, yaw)を設定
arm_id = pybullet.loadURDF("../urdf/simple2d_arm.urdf",arm_start_pos, arm_start_orientation, useFixedBase=True) # ロボットが倒れないように、useFixedBase=Trueでルートのリンクを固定

# GUIモードの際のカメラの位置などを設定
camera_distance = 1.5
camera_yaw = 180.0 # deg
camera_pitch = -10 # deg
camera_target_position = [0.0, 0.0, 1.0]
pybullet.resetDebugVisualizerCamera(camera_distance, camera_yaw, camera_pitch, camera_target_position)

b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:

b3Printf: No inertial data for link, using mass=1, localinertiadiagonal = 1,1,1, identity local inertial frame
b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:

b3Printf: target_position_vertual_link


### <a id='toc1_1_4_'></a>[2次元平面における同次変換行列の定義](#toc0_)


2次元平面におけるロボットアームでは、一般に
- リンク座標系はリンクの長さ方向に$x$軸をとります。
![](../images/RobotArm/robot_arm_homogeneous_matrix_forward_kinematics/2d_robot_arm_link_cordinate.png)

<br>

そのため、２次元平面におけるロボットアームの場合
1. 原点周りに $\theta$ 回転（回転行列）
2. $x$ 軸方向（リンクの長さ方向）に 「リンク長$l$」 だけ平行移動する（平行移動行列）

といった手順で座標変換することができます。
![](../images/RobotArm/robot_arm_homogeneous_matrix_forward_kinematics/2d_robot_arm_link_homogeneous_matrix_forward_kinematics.gif)

下記コードでは、2次元平面における「回転行列`Hz`」と「平行移動行列`Hp`」を定義しています。  

In [4]:
import numpy as np

def Hz(theta):
    """
    2次元平面における回転行列を求める（2次元平面における回転はz軸周り）

    Parameters
    ----------
    theta : float
        回転角度[rad]

    Returns
    -------
    numpy.ndarray
        回転行列
    """
    return np.array([[np.cos(theta), -np.sin(theta), 0], 
                     [np.sin(theta), np.cos(theta), 0], 
                     [0, 0, 1]])

def Hp(x, y):
    """
    2次元平面における平行移動行列を求める

    Parameters
    ----------
    x : float
        x方向の平行移動量
    y : float
        y方向の平行移動量

    Returns
    -------
    numpy.ndarray
        平行移動行列
    """
    return np.array([[1, 0, x], 
                     [0, 1, y], 
                     [0, 0, 1]])

また、「回転行列`Hz`」と「平行移動行列`Hp`」を組み合わせることで、上記の「原点周りの回転」「平行移動」を同時に行う行列「同次変換行列`H`」を定義することができます。


**【pythonコード例】**
```python
H = Hz(theta) @ Hp(LINK_LENGTH, 0) # 回転行列と平行移動行列を組み合わせて、同次変換行列を作成
```


<br>

なお、同次変換行列は「チェーンルール」により
- **リンク1座標系** → リンク2座標系 への同次変換行列`H12`
- リンク2座標系 → **エンドエフェクタ座標系** への同次変換行列`H2e`
  
を「**リンク1座標系** → **エンドエフェクタ座標系** への同次変換行列`H1e`」として、一つの同次変換行列にまとめることができます。


**【pythonコード例】**

```python
H12 = Hz(theta1) @ Hp(LINK1_LENGTH, 0) # リンク1座標系 → リンク2座標系 への同次変換行列
H2e = Hz(theta2) @ Hp(LINK2_LENGTH, 0) # リンク2座標系 → エンドエフェクタ座標系 への同次変換行列
H1e = H12 @ H2e # リンク1座標系 → エンドエフェクタ座標系 への同次変換行列
```

![](../images/RobotArm/robot_arm_homogeneous_matrix_forward_kinematics/direct_transformation.gif)


※ 軸数が増えていく場合も同様に
- 「**リンク1座標系** → リンク2座標系 への同次変換行列`H12`」
- 「リンク2座標系 → リンク3座標系 への同次変換行列`H23`」
- ...
- 「リンクn座標系 → **エンドエフェクタ座標系** への同次変換行列`Hne`」

を

「**リンク1座標系** → **エンドエフェクタ座標系** への同次変換行列`H1e`」として、一つの同次変換行列にまとめることができます。

<br>

### <a id='toc1_1_5_'></a>[同次変換行列による順運動学の実行](#toc0_)

以下コードを実行すると、「2軸ロボットアームの同次変換行列による順運動学」が計算され、pybullet上に結果（エンドエフェクタの位置）が表示されます。

```python
※ 画面上に表示される「エンドエフェクタの位置」は「ロボットアームの根本（=link1座標系）を原点とした座標位置」であることに注意してください（pybulletのワールド座標系を原点とした位置ではありません）。



In [11]:
LINK1_LENGTH = 0.5   # link1の長さ(「simple2d_arm.urdf」のlink1のz方向の長さ)
LINK2_LENGTH = 0.55  # link2の長さ(「simple2d_arm.urdf」のlink2+force_sensor_linkの長さ)

# ここの値を変えると、結果が変わります##########
# 各リンクの回転角度を定義
link1_angle_deg = 60
link2_angle_deg = 40
############################################

link1_angle_rad  = np.deg2rad(link1_angle_deg)
link2_angle_rad  = np.deg2rad(link2_angle_deg)

# リンク1座標系 -> リンク2座標系への変換行列
H12 = Hz(link1_angle_rad) @ Hp(LINK1_LENGTH, 0) # T12：link「1」座標系 -> link「2」座標系
# リンク2座標系 -> エンドエフェクタ座標系への変換行列
H2e = Hz(link2_angle_rad) @ Hp(LINK2_LENGTH, 0) # T2e：link「2」座標系 ->「e」nd effector座標系

# 「link1座標系 -> エンドエフェクタ座標系に変換する同時変換行列H12e」をあらかじめ定義
H1e = H12@H2e # T1e：link「1」座標系 ->「e」nd effector座標系

x1, y1 = 0, 0 # link1座標系の原点
# H12eを使って、「link1座標系の原点」から見た時の、end effector座標系の位置を求める
oe = H1e@np.array([x1, y1, 1])
xe, ye = oe[0], oe[1]
print("(xe, ye)=", xe, ye)

# 画面上に結果を表示
text_position = [0.5, 0.0, 2.0]
life_time = 10.0 # 表示期間（秒）
pybullet.addUserDebugText(f"x, y = ({xe:.2f}, {ye:.2f})", text_position, textSize=2, lifeTime=life_time)

# 実際に関節を動かして、順運動学の結果と等しいかを確認
pybullet.resetJointState(arm_id, 0, link1_angle_rad )
pybullet.resetJointState(arm_id, 1, link2_angle_rad )
pybullet.stepSimulation()

(xe, ye)= 0.15449350228318842 0.9746569660489338


()